In [11]:
# 최신순으로 최근 50개 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time

In [13]:
cafeName = '팡팡팡'
cafeNum = '053-252-2025'
cafeID = ""

In [14]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [15]:
cafeID = findCafe(cafeNum, cafeName)
cafeID

'1347974588'

In [16]:
finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'


In [17]:
driver = webdriver.Chrome()
driver.get(finalurl)

In [18]:
i=0
while i<5: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP) 
            time.sleep(3) 
            
            date = driver.find_element(By.CSS_SELECTOR, '.eCPGL > li:nth-last-child(1) > div:nth-last-child(1) > span > span:nth-last-child(1)').text
            date = date.split(" ")
            year = date[0][:-1]
            month = date[1]
            day = date[2]
            weekday = date[3]
            print(date)
            
            driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click() 
            time.sleep(3)
            i+=1
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 

2022년 12월 3일 토요일
2022년 11월 26일 토요일


KeyboardInterrupt: 

In [25]:
from datetime import datetime

date = '2022년 12월 3일 토요일'
date = date.split(" ")
year = int(date[0][:-1])
month = int(date[1][:-1])
day = int(date[2][:-1])
# print(year, month, day, weekday)
now = datetime.now()
dt = datetime(year, month, day)
print(now-dt > '30 days')

TypeError: '>' not supported between instances of 'datetime.timedelta' and 'str'

In [26]:
html = driver.page_source 
soup = BeautifulSoup(html, 'lxml')

In [ ]:
soup

In [28]:
one_review = soup.select('.YeINN')
review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
print('리뷰 총 개수 : '+str(review_num))

리뷰 총 개수 : 40


In [ ]:
try: 
    one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
    review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
    print('리뷰 총 개수 : '+str(review_num)) 
    
    # 모든 리뷰에 대해서 정보 수집
    for i in range(len(one_review)): 
        
        # user url
        user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
        print('user_url = '+user_url) 
        
        # user url로부터 user code 뽑아내기
        user_code = re.findall(r"my/(\w+)", user_url)[0]
        print('user_code = '+user_code) 
        
        # review 1개에 대한 id 만들기 
        res_code = re.findall(r"restaurant/(\d+)", thisurl)[0] 
        review_id = str(res_code)+"_"+user_code
        print('review_id = '+review_id) 
        
        # rating, 별점 
        rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
        print('rating = '+rating) 
        
        ## 주의
        ## 사진 리뷰 유무에 따라 날짜 파싱코드가 다르기 때문에
        ## case 별로 처리해줘야 함
        ## ('span', attrs = {'class':'_3WqoL'}) 
        ## 사진 없는 경우 : 총 6개 중 4번째 
        ## 사진 있는 경우 : 총 5개 중 3번째 
            
        # date 
        # 사진 리뷰 없음 
        if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5: 
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text 

        elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6: 
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text

        else: 
            date = ""
        
        print('date = '+date) 

        
        # review 내용
        try: 
            review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
        except: # 리뷰가 없다면
            review_content = "" 
        print('리뷰 내용 : '+review_content) 
        

        # image 내용
        sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'}) 
        
        if (sliced_soup != None): 
            sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'}) 
            
            try: 
                img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0] 
            
            except : 
                if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0): 
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0): 
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                else : 
                    img_url = ""
        
        else: 
            img_url = "" 
        
        
        print('이미지 url : '+img_url) 
        print('----------------------------------------------') 
        print('\n') 
        
        
        # 리뷰정보 
        # user_review_id
        user_review_id[restaurant_name][user_code] = review_id 
        
        # review_json 
        review_json[restaurant_name][review_id] = review_content 
        
        # image_json 
        image_json[restaurant_name][review_id] = img_url 
        
        # rating_df_list 
        naver_review = user_code, restaurant_name, rating, date
        rating_list.append(naver_review) 

# 리뷰가 없는 경우        
except NoSuchElementException: 
    none_review = "네이버 리뷰 없음" 
    print(none_review)
    review_num = 0 
    
    # 리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review 
    
    # rating_df_list
    naver_review = user_code, restaurant_name, none_review, none_reivew
    rating_list.append(naver_review)

In [ ]:
url = "https://pcmap-api.place.naver.com/graphql"

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.207 Whale/3.17.145.18 Safari/537.36", 
    "referer": "https://pcmap.place.naver.com/restaurant/1347974588/review/visitor",
    "authority": "pcmap-api.place.naver.com",
    "origin": "https://pcmap.place.naver.com",
    "sec-ch-ua": '"Whale";v="3", " Not;A Brand";v="99", "Chromium";v="106"',
    "sec-fetch-site": "same-site"
}

In [ ]:
params = [{"operationName":"getVisitorReviews",
           "variables":{
                "input":{
                    "businessId":"1347974588",
                    "businessType":"restaurant",
                    "item":"0",
                    "bookingBusinessId":None,
                    "page":3,
                    "size":10,
                    "isPhotoUsed":False,
                    "includeContent":True,
                    "getUserStats":True,
                    "includeReceiptPhotos":True,
                    "cidList":["220036","220052","220563"]
                },
                "id":"1347974588"
            },
            "query":"query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
}]

In [ ]:
getVisitorReviews($input: VisitorReviewsInput) {
    visitorReviews(input: $input) {
        items {
            id rating author {
                id nickname from imageUrl objectId url review {
                    totalCount imageCount avgRating __typename
                }
                theme {
                    totalCount
                    __typename
                }
                __typename
            }
            body thumbnail media {
                type thumbnail class __typename
            }
            tags status visitCount viewCount visited created reply {
                editUrl body editedBy created replyTitle __typename
            }
            originType item {
                name code options __typename
            }
            language highlightOffsets apolloCacheId translatedText businessName showBookingItemName showBookingItemOptions bookingItemName bookingItemOptions votedKeywords {
                code iconUrl iconCode displayName __typename 
            }
            userIdno isFollowing followerCount followRequested loginIdno receiptInfoUrl __typename
        }
        starDistribution {
            score count __typename
        }
        hideProductSelectBox total showRecommendationSort itemReviewStats {
            score count itemId starDistribution {
                score count __typename 
            }
            __typename
        }
        __typename
    }
}

In [ ]:
response = requests.post(url, params)
response

<Response [429]>

In [ ]:
dom = BeautifulSoup(response.text, "html.parser") 

In [ ]:
dom.select('script')

In [ ]:
reviews = dom.find_all('div', attrs = {'class':'_1Z_GL'})


In [ ]:
reviews

[]

In [ ]:
elements = dom.select(".eCPGL > li")
len(elements)

0

In [ ]:
"""


정보 뭐뭐 빼올지
    - 네이버: 날짜, 태그, ...
        ㄴ 키워드는 통계로만 받아도 되구,,
    - 카카오: 
    - 구글:

-
레드마인 내용 추가
레퍼 찾기 !!

카페 / 메뉴 호불호
키워드 << 좋은 키워드 나쁜 키워드 top 10

"""